# **Lab 3: Diagonalizable Matrix**
### **Họ và tên:** Đỗ Tiến Đạt
### **MSSV:** 23120119
### **Lớp:** 23CTT2

# Một số lưu ý trước khi thực thi mã nguồn
- Phải thực thi phần cell code chứa `class Matrix` và `class Vector` trước tiên để mã nguồn không bị lỗi.

In [ ]:
import random

class Matrix:
  def __init__(self, data: list[list[float]]):
    self.data = data
    self.row = len(data)
    self.col = len(data[0]) if data else 0

  def add(self, other: 'Matrix') -> 'Matrix':
    if self.row != other.row or self.col != other.col:
      raise ValueError('Matrix dimensions must match for addition.')
    return Matrix([
      [self.data[i][j] + other.data[i][j] for j in range(self.col)]
      for i in range(self.row)
    ])

  def subtract(self, other: 'Matrix') -> 'Matrix':
    if self.row != other.row or self.col != other.col:
      raise ValueError('Matrix dimensions must match for subtraction.')
    return Matrix([
      [self.data[i][j] - other.data[i][j] for j in range(self.col)]
      for i in range(self.row)
    ])

  def multiply(self, other: 'Matrix') -> 'Matrix':
    if self.col != other.row:
      raise ValueError('Matrix multiplication requires compatible dimensions.')
    return Matrix([
      [sum(self.data[i][k] * other.data[k][j] for k in range(self.col))
        for j in range(other.col)]
      for i in range(self.row)
    ])

  def multiply_scalar(self, scalar: float) -> 'Matrix':
    return Matrix([
      [self.data[i][j] * scalar for j in range(self.col)]
      for i in range(self.row)
    ])

  def transpose(self) -> 'Matrix':
    return Matrix([
      [self.data[i][j] for i in range(self.row)]
      for j in range(self.col)
    ])

  def get_minor(self, row, col) -> 'Matrix':
    return Matrix([
      [self.data[i][j] for j in range(self.col) if j != col]
      for i in range(self.row) if i != row
    ])

  def determinant(self) -> float:
    if (self.row != self.col):
      raise ValueError('Cannot find determinant')
    n = self.row
    if n == 1:
      return self.data[0][0]
    if n == 2:
      return self.data[0][0] * self.data[1][1] - self.data[0][1] * self.data[1][0]
    det = 0
    for i in range(n):
      cofactor = ((-1) ** i) * self.data[i][0]
      minor = self.get_minor(i, 0)
      det += cofactor * minor.determinant()
    return det

  def inverse(self) -> 'Matrix':
    if self.row != self.col:
      raise ValueError('Matrix must be square to invert.')

    n = self.row
    A_ext = [self.data[i] + [float(i == j) for j in range(n)] for i in range(n)]

    for i in range(n):
      # find pivot differ with 0
      if A_ext[i][i] == 0:
        for j in range(i + 1, n):
          if A_ext[j][i] != 0:
            A_ext[i], A_ext[j] = A_ext[j], A_ext[i]
            break
        else:
          raise ValueError('Matrix is not invertible.')

      pivot = A_ext[i][i] # normalize the pivot row
      A_ext[i] = [x / pivot for x in A_ext[i]]

      for j in range(n):
        if j != i:
          factor = A_ext[j][i]
          A_ext[j] = [a - factor * b for a, b in zip(A_ext[j], A_ext[i])]

    # result
    I = [row[n:] for row in A_ext]
    return Matrix(I)

  def gauss_jordan(self) -> 'Matrix':
    m = self.row
    n = self.col
    A_copy = Matrix([row[:] for row in self.data])
    eps = 1e-12

    r = 0
    for c in range(n):  # column pointer
      # Find the row with the largest value in column c at or below row r
      pivot_row = None
      max_val = eps
      for i in range(r, m):
        if abs(A_copy.data[i][c]) > max_val:
            max_val = abs(A_copy.data[i][c])
            pivot_row = i
      if pivot_row is None:
        continue  # skip this column, no pivot found
      # Swap current row with pivot_row
      if pivot_row != r:
        A_copy.data[r], A_copy.data[pivot_row] = A_copy.data[pivot_row], A_copy.data[r]
      # Normalize the pivot row
      pivot = A_copy.data[r][c]
      A_copy.data[r] = [x / pivot for x in A_copy.data[r]]
      # Eliminate all other rows in column c
      for i in range(m):
        if i != r:
            factor = A_copy.data[i][c]
            A_copy.data[i] = [a - factor * b for a, b in zip(A_copy.data[i], A_copy.data[r])]
      r += 1
      if r == m:
        break
    return A_copy

  def extract_col(self, col_idx: int) -> 'Vector':
    if col_idx < 0 or col_idx >= self.col:
      raise ValueError('Invalid column index.')
    return Vector([self.data[i][col_idx] for i in range(self.row)])

  def subtract_lambda(self, lamb : float) -> 'Matrix':
    return Matrix([
      [self.data[i][j] - lamb if i == j else self.data[i][j] for j in range(self.col)]
      for i in range(self.row)
    ])

  def __str__(self) -> str:
    return '\n'.join(['\t'.join(f'{x:.3f}' for x in row) for row in self.data])


class Vector(Matrix):
  def __init__(self, data: list[float]):
    super().__init__([[x] for x in data])
    self.length = len(data)

  @property
  def data_flat(self) -> list[float]:
    return [row[0] for row in self.data]

  def norm(self) -> float:
    return sum(x ** 2 for x in self.data_flat) ** 0.5

  def dot(self, other: 'Vector') -> float:
    if self.length != other.length:
      raise ValueError('Dot product requires same length vectors.')
    return sum(self.data_flat[i] * other.data_flat[i] for i in range(self.length))

  def multiply_scalar(self, scalar: float) -> 'Vector':
    return Vector([x * scalar for x in self.data_flat])

  def add(self, other: 'Vector') -> 'Vector':
    if self.length != other.length:
      raise ValueError('Vector lengths must match for addition.')
    return Vector([
      self.data_flat[i] + other.data_flat[i]
      for i in range(self.length)
    ])

  def subtract(self, other: 'Vector') -> 'Vector':
    if self.length != other.length:
        raise ValueError('Vector lengths must match for subtraction.')
    return Vector([
        self.data_flat[i] - other.data_flat[i]
        for i in range(self.length)
    ])

  @staticmethod
  def random_unit_vector(dim: int) -> 'Vector':
    while True:
      data = [random.uniform(-1, 1) for _ in range(dim)]
      norm = sum(x ** 2 for x in data) ** 0.5
      if norm > 1e-8:
          return Vector([x / norm for x in data])

  def __str__(self) -> str:
    return '[' + ', '.join(f'{x:.3f}' for x in self.data_flat) + ']'


In [ ]:
A = [[2, 0, -2],
     [0, 3, 0],
     [0, 0, 3]]

A = Matrix(A)
print(A.inverse())

0.500	0.000	0.333
0.000	0.333	0.000
0.000	0.000	0.333


#I. Chéo hóa ma trận
- **Đa thức đặc trưng, phương trình đặc trưng**

  - Cho $A$ là ma trận vuông cấp $n$ trường số $\mathbb{K}$. Khi ấy, $det(A - λI)$ được gọi là đa thức đặc trưng của $A$.
  - Kí hiệu $P_A(λ) = |A - λI_n|$, khi đó $P_A(λ) = 0$ được gọi là phương trình đặc trưng của ma trận $A$.
- **Trị riêng, vector riêng**
  - Số $λ \in \mathbb{K}$ được gọi là trị riêng của ma trận $A$, nếu $\exists \vec{x} \in \mathbb{K}^{n}$, $\vec{x} \neq \vec{0}$ sao cho
  \begin{equation}
    A\vec{x} = λ\vec{x}
  \end{equation}
  - $\vec{x}$ được gọi là vector riêng của ma trận $A$ ứng với trị riêng $λ$
  - $λ$ là trị riêng của ma trận $A$ khi và chỉ khi $λ$ là nghiệm của $P_A(λ) = 0$
- **Các bước tìm trị riêng, vector riêng**
  - **Bước 1:** Giải phương trình đặc trưng $P_A(λ) = 0$ để tìm trị riêng của ma trận A.
  - **Bước 2:** Tìm vector riêng ứng với trị riêng $λ$ bằng cách giải hệ phương trình tuyến tính $(A - λI_n)\vec{x} = \vec{0}$. Nghiệm không tầm thường (khác $\vec{0}$) chính là các vector riêng cần tìm.
- **Ma trận chéo hóa**
  - Ma trận vuông $A$ được gọi là chéo hóa được nếu tồn tại một ma trận đường chéo $D$ và ma trận khả nghịch $P$ sao cho:
  \begin{equation}
    A = PDP^{-1}
  \end{equation}
  - Trong đó ma trận đường chéo $D$ có dạng

  \begin{pmatrix}
  λ_1   & 0     & \dots & 0 \\
  0     & λ_2   & \dots & 0 \\
  \dots & \dots & \dots & \dots \\
  0     & 0     & \dots & λ_n
  \end{pmatrix}
- **Giải thuật chéo hóa ma trận:** Cho $A$ là ma trận vuông cấp $n$. Các bước để chéo hóa ma trận:
  - **Bước 1:** Tìm đa thức đặc trưng của ma trận $A$: $P_A(λ) = |A - λI_n|$. Nếu $P_A(λ)$ không có nghiệm trên $\mathbb{K}$ thì $A$ không chéo hóa được và giải thuật kết thúc. Ngược lại, chuyển sang **bước 2**.
  - **Bước 2:** Tìm tất cả các nghiệm $λ_1, λ_2, \dots, λ_k$ của đa thức đặc trưng $P_A(λ)$ và có bội đai số tương ứng là $α_1, α_2, \dots, α_k$. Đối với mỗi $λ_i$ tìm số chiều của không gian nghiệm $E(λ_i)$ của hệ phương trình $(A - λI_n)\vec{x} = \vec{0}$. Nếu tồn tại $i \in  \overline{1, k}$ thỏa $dim(E(λ_i)) \neq α_i$ thì $A$ không chéo hóa được và giải thuật kết thúc. Ngược lại thì $A$ chéo hóa được và chuyển sang **bước 3**
  - **Bước 3:** Với mỗi $i \in  \overline{1, k}$ ta tìm một cơ sở $B_i$ cho $E(λ_i)$, đặt $P$ là ma trận có được bằng cách sắp xếp các vector của $B_i$, $i \in  \overline{1, k}$. Khi đó ma trận $P$ làm chéo $A$ và $P^{-1}AP$ là ma trận đường chéo D.
  \begin{equation}
    D = diag(λ_1,\dots,λ_1,\dots,λ_k, \dots, λ_k) \\
    D = P^{-1}AP
  \end{equation}
    - Trong đó: $λ_1$ xuất hiện $α_1$ lần, ... , $λ_k$ xuất hiện $α_k$ lần.

##1. Phương pháp đặc trưng


### 1.1 Tìm các trị riêng
Dùng giải thuật **tìm kiếm nhị phân** kết hợp với **định lí giá trị trung gian** để xấp xỉ nghiệm của phương trình đặc trưng $P_A(λ)$ trong đoạn $[a, b]$.
1. **Khởi tạo**: Cho $\lambda = a$ và $\lambda_{next} = a + c \times step$.
2. **Tính định thức**: $x = \det(A - \lambda I_n)$, $y = \det(A - \lambda_{next} I_n)$.
3. **Kiểm tra dấu**: Nếu $xy < 0$ ($*$), tức là $x$ và $y$ trái dấu, nghĩa là tồn tại ít nhất một nghiệm (trị riêng) trong đoạn $[\lambda, \lambda_{next}]$ do định lí giá trị trung gian.
4. **Tìm phân đoạn chứa nghiệm**: Nếu ($*$) đúng, áp dụng **giải thuật tìm kiếm nhị phân (bisection)** để chia nhỏ đoạn $[\lambda, \lambda_{next}]$ và tìm nghiệm với độ chính xác mong muốn.


In [ ]:
def find_all_eigenvalues(A: Matrix, search_min=-1000, search_max=1000, step=0.01, eps=1e-10)->list:
  n = A.row
  found = []
  last_det = A.subtract_lambda(search_min).determinant()
  for i in range(int((search_max-search_min)/step)):
    lamb = search_min + i * step
    det = A.subtract_lambda(lamb).determinant()
    if last_det * det < 0:  # Root between lamb-step and lamb
      # refine by bisection
      a, b = lamb - step, lamb
      for _ in range(30):
        m = (a + b) / 2
        dm = A.subtract_lambda(m).determinant()
        if abs(dm) < eps:
          break
        if A.subtract_lambda(a).determinant() * dm < 0:
          b = m
        else:
          a = m
      root = (a + b) / 2
      if all(abs(root - x) > 1e-3 for x in found):
        found.append(root)
    last_det = det
  # try integer roots if missed by numeric
  if len(found) < n:
    for val in range(int(search_min), int(search_max)+1):
      det = A.subtract_lambda(val).determinant()
      if abs(det) < eps and all(abs(val - x) > 1e-3 for x in found):
        found.append(float(val))
  found.sort()
  # Remove near duplicates
  result = []
  for x in found:
    if not result or abs(x-result[-1]) > 1e-3:
      result.append(x)
  return result[:n]

### 1.2 Tìm các vector riêng

Sau khi đã tìm được các trị riêng $\lambda$ của ma trận $A$, ta tìm các **vector riêng** tương ứng bằng cách giải hệ phương trình:
\begin{equation}
(A - \lambda I)\vec{x} = \vec{0}
\end{equation}

#### Các bước thực hiện:

1. **Với mỗi trị riêng $\lambda$ đã tìm được:**
   - Tạo ma trận $A - \lambda I$.
   - Giải hệ phương trình đồng nhất $(A - \lambda I)\vec{x} = \vec{0}$ để tìm nghiệm không tầm thường $\vec{x} \neq 0$.

2. **Cách giải hệ phương trình:**
   - Đưa ma trận $A - \lambda I$ về dạng bậc thang rút gọn (Gauss-Jordan).
   - Tìm cơ sở của không gian nghiệm (null space) của ma trận này.
   - Mỗi vector cơ sở thu được chính là một vector riêng ứng với trị riêng $\lambda$.

In [ ]:
def find_null_space(A: Matrix, eps=1e-10) -> list:
  n_rows, n_cols = A.row, A.col
  pivot_cols = []
  row_pivots = [-1] * n_rows
  for i in range(n_rows):
    for j in range(n_cols):
        if abs(A.data[i][j]) > eps:
          pivot_cols.append(j)
          row_pivots[i] = j
          break
  free_cols = [j for j in range(n_cols) if j not in pivot_cols]
  if not free_cols:
    return []
  basis = []
  for free in free_cols:
    vec = [0.0] * n_cols
    vec[free] = 1.0
    for i in reversed(range(n_rows)):
      pivot = row_pivots[i]
      if pivot == -1:
          continue
      s = 0.0
      for j in range(pivot + 1, n_cols):
          s += A.data[i][j] * vec[j]
      vec[pivot] = -s / A.data[i][pivot]
    cleaned = [0 if abs(x) < eps else float(x) for x in vec]
    # Only add nonzero vector
    if any(abs(x) > eps for x in cleaned):
      basis.append(Vector(cleaned))
  return basis

def find_all_eigenvectors(A: Matrix, eigenvalues: list[float], eps=1e-8) -> list:
  results = []
  for lamb in eigenvalues:
    A_l = A.subtract_lambda(lamb)
    A_gj = A_l.gauss_jordan()
    eigvecs = find_null_space(A_gj, eps=eps)
    results.append((lamb, eigvecs))
  return results

In [ ]:
# Example matrix
matrix_data = [[3, -2, 0],
               [-2, 3, 0],
               [0, 0, 5]]
A = Matrix(matrix_data)

eigenvalues = find_all_eigenvalues(A)
eigenvectors_list = find_all_eigenvectors(A, eigenvalues)

for lamb, eigenvectors in eigenvectors_list:
  print(f'Eigenvalue: {lamb}')
  for vec in eigenvectors:
    print(f'Eigenvector: {vec}')

Eigenvalue: 1.0
Eigenvector: [1.000, 1.000, 0.000]
Eigenvalue: 5.0
Eigenvector: [-1.000, 1.000, 0.000]
Eigenvector: [0.000, 0.000, 1.000]


In [ ]:
# Example matrix
matrix_data = [[3, -2, 0],
               [-2, 3, 0],
               [0, 0, 4]]
A = Matrix(matrix_data)

eigenvalues = find_all_eigenvalues(A)
eigenvectors_list = find_all_eigenvectors(A, eigenvalues)

for lamb, eigenvectors in eigenvectors_list:
  print(f'Eigenvalue: {lamb}')
  for vec in eigenvectors:
    print(f'Eigenvector: {vec}')

Eigenvalue: 1.0
Eigenvector: [1.000, 1.000, 0.000]
Eigenvalue: 4.0
Eigenvector: [0.000, 0.000, 1.000]
Eigenvalue: 5.0
Eigenvector: [-1.000, 1.000, 0.000]


In [ ]:
matrix_data = [[2, 0, -2],
              [0, 3, 0],
              [0, 0, 3]]
A = Matrix(matrix_data)

eigenvalues = find_all_eigenvalues(A)
eigenvectors_list = find_all_eigenvectors(A, eigenvalues)

for lamb, eigenvectors in eigenvectors_list:
    print(f'Eigenvalue: {lamb}')
    for vec in eigenvectors:
        print(f'Eigenvector: {vec}')

Eigenvalue: 2.0
Eigenvector: [1.000, 0.000, 0.000]
Eigenvalue: 3.0
Eigenvector: [0.000, 1.000, 0.000]
Eigenvector: [-2.000, 0.000, 1.000]


## 2. Phương pháp lặp



### 2.1 The Power Method
- Ta sẽ tiến hành thực hiện lặp đi lặp lại một phương pháp cho đến khi trị riêng và vector riêng hội tụ (bé hơn 1 sai số cho trước)
- Có một số phương pháp để tìm trị riêng như là : **The Power Method, The Shifted Power Method, ...**Ở đây ta sẽ chỉ đi qua **The Power Method**
---
- **The Power Method** là một cách để tìm trị riêng có giá trị tuyệt đối lớn nhất ứng với trị riêng của nó.
- Giải thuật lặp: Cho $A$ là ma trận vuông cấp $n$ hệ số thực, các bước thực hiện như sau:
  - **Bước 1:** Chọn 1 vector đơn vị bất kì $u_0$
  - **Bước 2:** Tạo một dãy các n-vector đơn vị bất kì $u_1, u_2, \dots$ bằng cách lặp lại các bước **2.1** đến **2.4** cho đến khi một trong các điều kiện dừng trong từng bước **2.3** hoặc **2.4** thỏa mãn, hoặc đến khi dãy các vector không hội tụ đáp án.
    - **2.1** Cho $u_{k-1}$, tính $w_k = Au_{k-1}$   
    - **2.2** Tính $u_k = \frac{w_k}{\||w_k\||}$
    - **2.3** Nếu $u_{k-1}$ bằng $u_k$ với độ chính xác mong muốn thì $λ = \||w_k\||$
    - **2.4** Nếu $u_{k-1}$ bằng $-u_k$ với độ chính xác mong muốn thì $λ = -\||w_k\||$
  - **Bước 3:** Vector $u_k$ cuối cùng được tính trong **bước 2** là một vector riêng xấp xỉ tương ứng với trị riêng $λ$

In [ ]:
def eigen_power_iteration(A: Matrix, n_iter = 10000, eps = 1e-10) -> tuple[Vector, float]:
  u = Vector.random_unit_vector(A.col) # choose randomly a unit vector
  lamb = 0
  for _ in range(n_iter):
    w = A.multiply(u) # w = A * u
    w = Vector([row[0] for row in w.data])  # convert it back to a Vector
    w_norm = w.norm()
    if w_norm == 0:
        return None
    u_next = w.multiply_scalar(1 / w_norm)

    # Check convergence by comparing each element
    if all(abs(u.data[i][0] - u_next.data[i][0]) < eps for i in range(u.length)):
      lamb = w_norm
      u = u_next
      break

    # Case where the vector converges in the opposite direction
    if all(abs(u.data[i][0] + u_next.data[i][0]) < eps for i in range(u.length)):
      lamb = -w_norm
      u = u_next
      break

    u = u_next
  return u, lamb

In [ ]:
matrix_data = [[3, -2, 0],
            [-2, 3, 0],
            [0, 0, 5]]
A = Matrix(matrix_data)

# Find the dominant eigenvector and eigenvalue
dominant_eigenvector, dominant_eigenvalue = eigen_power_iteration(A)
print('Dominant Eigenvector:', dominant_eigenvector)
print('Dominant Eigenvalue:', dominant_eigenvalue)

Dominant Eigenvector: [0.707, -0.707, 0.017]
Dominant Eigenvalue: 5.0


###2.2 Wielandt Deflation with One Vector
- Sau khi đã tìm được một trị riêng lớn nhất $λ_1$tương ứng với vector riêng $u_1$, vấn đề tiếp theo là **tính trị riêng có giá trị tuyệt đối lớn thứ 2 tiếp theo $λ_2$** cùng với **vector riêng $u_2$**. Có một kĩ thuật để tìm là **Deflation procedure**.
- Kĩ thuật này sẽ cập nhật ma trận ban đầu thành ma trận mới sao cho ma trận mới này triệt tiêu đi $λ_1$ mà vẫn giữ các trị riêng còn lại không thay đổi. Khi đó dùng phương pháp lặp để tìm trị riêng và vector riêng tiếp theo của ma trận ban đầu và tiếp tục như vậy cho đến khi tìm được hết tất cả trị riêng và vector riêng.
- Một số kĩ thuật là: **Wielandt Deflation with One Vector, Optimality in Wieldant's Deflation,...** Ta sẽ đi tìm hiểu kĩ thuật **Wielandt Deflation with One Vector**
---
#### **Một số khái niệm:**
Cho $A$ là ma trận vuông cấp $n$ với hệ số phức.
- **Ma trận liên hợp:** là ma trận thu được bằng cách lấy liên hợp phức với từng hệ số trong ma trận $A$, kí hiệu là $\overline{A}$
- **Ma trận chuyển vị liên hợp:** là ma trận thu được bằng cách lấy chuyển vị liên hợp của ma trận $A$, kí hiệu $A^H$ và $A^H = \overline{A}^T$

- **Vector riêng trái** và **vector riêng phải**: Nếu $λ$ là một trị riêng của $A$ thì $\overline{λ}$ là một trị riêng của $A^H$. Vector riêng $v$ ứng với $\overline{λ}$ của $A^H$ là vector riêng trái của $A$, còn vector riêng $u$ ứng với $λ$ của $A$ là vector riêng phải của $A$.
\begin{equation}
  Au = λu \\
  A^Hv = \overline{λ}v
\end{equation}
#### Wielandt Deflation with One Vector
- Quy tắc cập nhập ma trận là:
\begin{equation}
  A_i := A_{i-1} - σ u_{i-1} v_{i-1}^H, \space ∀i = \overline{1,\dots ,k}
\end{equation}
- Trong đó:
  - $\sigma$ là hệ số dịch thích hợp.
  - $u_i$ là vector riêng phải của ma trận $A_i$
  - $v_i$ là vector tùy ý thỏa $v_{i-1}^H u_{i-1} = 1$
- Phép biến đổi trên chỉ thay đổi một trị riêng duy nhất $λ_{i-1}$ thành $λ_{i-1} - σ$ mà vẫn giữ nguyên các trị riêng khác.

- **Định lý Wielandt** Các trị riêng của $A$ sau khi cập nhật (gọi là $A_1$) là
  \begin{equation}
    S(A_1) = \{λ_1 - σ, λ_2, \dots, λ_k\}
  \end{equation}

- Do đó để triệt tiêu $λ_{i-1}$ sau khi cập nhật ta sẽ chọn $σ = λ_{i-1}$
- Có nhiều cách để chọn vector $v$, một trong những cách đơn giản nhất là chọn vector riêng trái của $A$ (gọi là $w$) tức là $v = w$, khi xét $A$ là ma trận hệ số thực thì $A^H = A^T$ và $w^H = w^T$.
- Quy tắc cập nhật trên trở thành:
\begin{equation}
  A_i := A_{i-1} - \frac{λ_{i-1}}{⟨u_{i-1}, w_{i-1}⟩} u_{i-1} w_{i-1}^T, \space ∀i = \overline{1,\dots ,k}
\end{equation}


In [ ]:
def eigens_deflate(A: Matrix, n_iter=10000, eps=1e-10) -> tuple[list, list]:
  eigenvectors = []
  eigenvalues = []
  A_copy = A
  for _ in range(A.row):
    # Right eigenvector
    u, lamb = eigen_power_iteration(A_copy, n_iter, eps)
    if u is None or lamb is None:
        break
    # Left eigenvector (from A^T)
    AT = A_copy.transpose()
    w, _ = eigen_power_iteration(AT, n_iter, eps)
    if w is None:
      break
    dot_uw = w.dot(u)
    # Deflation: A = A - λ * u * w^T
    uwT = (u.multiply(w.transpose()))  # u (n×1) * w^T (1×n) = (n×n)
    uwT = uwT.multiply_scalar(1 / dot_uw) # Normalize w so that u^T * w = 1
    A_copy = A_copy.subtract(uwT.multiply_scalar(lamb))
    # Save the result
    eigenvectors.append(u)
    eigenvalues.append(lamb)

  return eigenvectors, eigenvalues


In [ ]:
matrix_data = [[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]]
A = Matrix(matrix_data)
B, C = eigens_deflate(A)
print('Eigenvectors:')
for b in B:
  print(b)
print('\nEigenvalues')
print(C)

Eigenvectors:
[-0.111, 0.111, 0.988]
[0.579, -0.579, 0.575]
[0.707, 0.707, -0.000]

Eigenvalues
[5.0, 4.999999999999999, 1.0]


##3. Chéo hóa ma trận
- Từ danh sách các trị riêng ta sẽ tiến hành ghép cặp tương ứng với các vector riêng theo thứ tự giảm dần các trị riêng.
- Từ các trị riêng ta sẽ dựng ma trận đường chéo $D$ với các phần tử nằm trên đường chéo chính theo đúng thứ tự trong danh sách các trị riêng.
- Từ danh sách các vector riêng ta đặt các cột là các vector riêng ứng tương ứng với trị riêng, ta thu được ma trận $P$.
- Cuối cùng, lấy nghịch đảo ma trận P ta sẽ thu được $P^{-1}$

Như vậy ta có phân tích $A = PDP^{-1}$

### Phương pháp đặc trưng

In [ ]:
def diag(A: Matrix) -> tuple[Matrix, Matrix, Matrix]:
  n = A.row

  # Find all eigenvalues and their corresponding eigenvectors
  eigenpairs = find_all_eigenvectors(A, find_all_eigenvalues(A))

  eig_list = []
  for lamb, vectors in eigenpairs:
    for v in vectors:
        eig_list.append((lamb, v))

  # Combine and sort eigenpairs in descending order of eigenvalues
  eig_list.sort(key=lambda x: x[0], reverse=True)
  eig_list = eig_list[:n]

  eigenvalues = [lamb for lamb, v in eig_list]
  eigenvectors = [v for lamb, v in eig_list]

  if len(eigenvectors) != n:
    raise ValueError("Matrix is not diagonalizable or insufficient independent eigenvectors.")

  # Create matrix P, D, inversed P from eigenvectors and eigenvalues
  P = Matrix([[eigenvectors[j].data_flat[i] for j in range(n)] for i in range(n)])
  D = Matrix([[eigenvalues[i] if i == j else 0 for j in range(n)] for i in range(n)])
  P_inv = P.inverse()

  return P, D, P_inv

In [ ]:
matrix_data = [[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]]
A = Matrix(matrix_data)
P, D, P_inv = diag(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

3.000	-2.000	0.000
-2.000	3.000	0.000
0.000	0.000	5.000

Eigenvector matrix P: 

-1.000	0.000	1.000
1.000	0.000	1.000
0.000	1.000	0.000

Diagonal matrix D:

5.000	0.000	0.000
0.000	5.000	0.000
0.000	0.000	1.000

Inverse of P (P^-1):

-0.500	0.500	0.000
0.000	0.000	1.000
0.500	0.500	0.000

Check diagonalization: 

3.000	-2.000	0.000
-2.000	3.000	0.000
0.000	0.000	5.000


In [ ]:
matrix_data = [[2, 0, -2],
              [0, 3, 0],
              [0, 0, 3]]
A = Matrix(matrix_data)
P, D, P_inv = diag(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

2.000	0.000	-2.000
0.000	3.000	0.000
0.000	0.000	3.000

Eigenvector matrix P: 

0.000	-2.000	1.000
1.000	0.000	0.000
0.000	1.000	0.000

Diagonal matrix D:

3.000	0.000	0.000
0.000	3.000	0.000
0.000	0.000	2.000

Inverse of P (P^-1):

0.000	1.000	0.000
0.000	0.000	1.000
1.000	0.000	2.000

Check diagonalization: 

2.000	0.000	-2.000
0.000	3.000	0.000
0.000	0.000	3.000


In [ ]:
matrix_data = [[-1, 3, -1],
              [-3, 5, -1],
              [-3, 3, 1]]
A = Matrix(matrix_data)
P, D, P_inv = diag(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

-1.000	3.000	-1.000
-3.000	5.000	-1.000
-3.000	3.000	1.000

Eigenvector matrix P: 

1.000	-0.333	1.000
1.000	0.000	1.000
0.000	1.000	1.000

Diagonal matrix D:

2.000	0.000	0.000
0.000	2.000	0.000
0.000	0.000	1.000

Inverse of P (P^-1):

-3.000	4.000	-1.000
-3.000	3.000	0.000
3.000	-3.000	1.000

Check diagonalization: 

-1.000	3.000	-1.000
-3.000	5.000	-1.000
-3.000	3.000	1.000


### Phương pháp lặp

In [ ]:
def diag_power_method(A: Matrix) -> tuple[Matrix, Matrix, Matrix]:
  eigenvectors, eigenvalues = eigens_deflate(A)

  # Combine and sort eigenpairs in descending order of eigenvalues
  eigen_pairs = list(zip(eigenvalues, eigenvectors))
  eigen_pairs.sort(reverse = True)

  eigenvalues = [x[0] for x in eigen_pairs]
  eigenvectors = [x[1] for x in eigen_pairs]

 # Create matrix P, D, inversed P from eigenvectors and eigenvalues
  P_data = [vec.data for vec in eigenvectors]  # each is [[x], [y], ...]
  P = Matrix([[vec[i][0] for vec in P_data] for i in range(A.row)])  # transpose while building
  D = Matrix([[eigenvalues[i] if i == j else 0 for j in range(A.col)] for i in range(A.row)])
  P_inv = P.inverse()

  return P, D, P_inv


In [ ]:
matrix_data = [[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]]
A = Matrix(matrix_data)
P, D, P_inv = diag_power_method(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

3.000	-2.000	0.000
-2.000	3.000	0.000
0.000	0.000	5.000

Eigenvector matrix P: 

-0.238	-0.705	0.707
0.238	0.705	0.707
-0.941	-0.077	-0.000

Diagonal matrix D:

5.000	0.000	0.000
0.000	5.000	0.000
0.000	0.000	1.000

Inverse of P (P^-1):

0.060	-0.060	-1.092
-0.729	0.729	0.369
0.707	0.707	-0.000

Check diagonalization: 

3.000	-2.000	0.000
-2.000	3.000	0.000
-0.000	-0.000	5.000


In [ ]:
matrix_data = [[2, 0, -2],
              [0, 3, 0],
              [0, 0, 3]]
A = Matrix(matrix_data)
P, D, P_inv = diag_power_method(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

2.000	0.000	-2.000
0.000	3.000	0.000
0.000	0.000	3.000

Eigenvector matrix P: 

-0.891	0.317	-1.000
-0.093	0.935	0.000
0.445	-0.158	-0.000

Diagonal matrix D:

3.000	0.000	0.000
0.000	3.000	0.000
0.000	0.000	2.000

Inverse of P (P^-1):

-0.000	0.394	2.328
0.000	1.108	0.231
-1.000	0.000	-2.000

Check diagonalization: 

2.000	0.000	-2.000
0.000	3.000	0.000
-0.000	0.000	3.000


In [ ]:
matrix_data = [[-1, 3, -1],
              [-3, 5, -1],
              [-3, 3, 1]]
A = Matrix(matrix_data)
P, D, P_inv = diag_power_method(A)

print('Original matrix A:\n')
print(A)
print('\nEigenvector matrix P: \n')
print(P)
print('\nDiagonal matrix D:\n')
print(D)
print('\nInverse of P (P^-1):\n')
print(P_inv)
print('\nCheck diagonalization: \n')
print(P.multiply(D.multiply(P_inv)))

Original matrix A:

-1.000	3.000	-1.000
-3.000	5.000	-1.000
-3.000	3.000	1.000

Eigenvector matrix P: 

-0.450	0.718	-0.577
-0.653	0.692	-0.577
-0.609	-0.078	-0.577

Diagonal matrix D:

2.000	0.000	0.000
0.000	2.000	0.000
0.000	0.000	1.000

Inverse of P (P^-1):

4.889	-5.054	0.164
0.279	1.011	-1.290
-5.196	5.196	-1.732

Check diagonalization: 

-1.000	3.000	-1.000
-3.000	5.000	-1.000
-3.000	3.000	1.000


## Ý tưởng thực hiện
1.**Tìm các trị riêng và vector riêng**
- **Phương pháp đặc trưng:**
  - **Bước 1:** Tính đa thức đặc trưng $P_A(\lambda) = \det(A - \lambda I)$ của ma trận $A$.
  - **Bước 2:** Tìm các nghiệm thực của đa thức đặc trưng này, tức là các $\lambda$ sao cho $P_A(\lambda) = 0$. Các nghiệm này chính là **các trị riêng** (eigenvalues) của ma trận $A$.
  - **Bước 3:** Với mỗi trị riêng $\lambda$, giải hệ phương trình đồng nhất $(A - \lambda I)x = 0$ để tìm các vector riêng (eigenvectors) tương ứng.
  - **Bước 4:** Lưu các trị riêng tương ứng với các vector riêng

- **Phương pháp lặp (Power Iteration & Deflation):**
  - **Bước 1:** Dùng thuật toán lặp (power iteration) để tìm trị riêng lớn nhất (theo giá trị tuyệt đối) của ma trận $A$ và vector riêng tương ứng:
    - Khởi tạo một vector đơn vị ngẫu nhiên $u$.
    - Lặp: $w = A u$, chuẩn hóa $w$ để thành vector đơn vị mới, kiểm tra hội tụ.
    - Khi hội tụ, $u$ là vector riêng, trị riêng tương ứng $\lambda$.
  - **Bước 2:** Dùng kỹ thuật deflation để "loại bỏ" trị riêng vừa tìm được ra khỏi ma trận, nhận ma trận mới $A'$ để tiếp tục tìm trị riêng/vectơ riêng tiếp theo:
    - $A' := A - \lambda u w^T$ (có thể dùng trái và phải eigenvector nếu ma trận không đối xứng).
    - Lưu các trị riêng và vector riêng sau mỗi lần lặp.
    - Lặp lại quá trình trên với $A'$ cho đến khi tìm đủ số trị riêng cần thiết.
---
2. **Chéo hóa ma trận**
- **Bước 1:** Từ danh sách các trị riêng ta sẽ tiến hành ghép cặp tương ứng với các vector riêng theo thứ tự giảm dần các trị riêng.
- **Bước 2:** Từ các trị riêng ta sẽ dựng ma trận đường chéo $D$ với các phần tử nằm trên đường chéo chính theo đúng thứ tự trong danh sách các trị riêng.
- **Bước 3:** Từ danh sách các vector riêng ta đặt các cột là các vector riêng ứng tương ứng với trị riêng, ta thu được ma trận $P$.
- **Bước 4:** Lấy nghịch đảo ma trận P ta sẽ thu được $P^{-1}$


## Mô tả các hàm sử dụng

**1.** `class Matrix` và `class Vector`  
Cung cấp các thuộc tính và phương thức để thực hiện các phép toán đại số tuyến tính cơ bản. Một số phương thức quan trọng:
- `inverse(self)`: Sử dụng giải thuật Gauss-Jordan để tìm ma trận nghịch đảo.
- `gauss_jordan(self)`: Đưa ma trận về dạng bậc thang rút gọn (reduced row echelon form).
- `subtract_lambda(self, lambda: float)`: Trả về ma trận $A - \lambda I$ bằng cách trừ $\lambda$ trên đường chéo chính.
- `determinant(self)`: Tính định thức ma trận (có thể dùng đệ quy hoặc phương pháp Laplace).

---

**2.** Tìm các trị riêng và các vector riêng  

**i. Phương pháp đặc trưng**
- `find_all_eigenvalues(A: Matrix)`: Tìm các nghiệm thực của phương trình đặc trưng $\det(A - \lambda I_n) = 0$, là các trị riêng (eigenvalues) của $A$.
- `find_null_space(A: Matrix, eps=1e-10)`: Tìm không gian nghiệm (null space) của ma trận, trả về cơ sở không gian nghiệm.
- `find_all_eigenvectors(A: Matrix, eigenvalues: list[float])`: Với mỗi trị riêng, giải $(A - \lambda I)\vec{x} = \vec{0}$ để tìm các vector riêng (eigenvectors) tương ứng.

**ii. Phương pháp lặp**
- `eigen_power_iteration(A: Matrix, n_iter=10000, eps=1e-10)`: Dùng phương pháp lặp để tìm trị riêng lớn nhất (theo trị tuyệt đối) và vector riêng tương ứng.
- `eigens_deflate(A: Matrix, n_iter=10000, eps=1e-10)`: Ứng dụng phương pháp "deflation" để lần lượt tìm nhiều trị riêng và vector riêng của $A$ bằng power iteration.

---

**3.** Chéo hóa ma trận
- `diag(A: Matrix)`: Thực hiện chéo hóa ma trận bằng cách tìm đủ $n$ vector riêng độc lập, trả về bộ ba $(P, D, P^{-1})$ với $D$ là ma trận chéo chứa các trị riêng, $P$ gồm các vector riêng, $P^{-1}$ là nghịch đảo của $P$.
- `diag_power_method(A: Matrix)`: Tương tự với `diag(A: Matrix)`, chỉ khác phương pháp tìm trị riêng với vector riêng

# II. Mở rộng

### Trong thư viện `numpy` và `scipy` có hỗ trợ các phương thức cho việc chéo hóa ma trận





- **`numpy.linalg.eig`**:  
  - Hàm này trả về trị riêng (eigenvalues) và vector riêng (eigenvectors) của một ma trận vuông.

In [ ]:
import numpy as np
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])

B, C = np.linalg.eig(A)
print(B)
print(C)

[5. 1. 5.]
[[ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


- **`scipy.linalg.eig`**:  
    - Hàm tương tự trong SciPy, hỗ trợ tốt hơn cho số phức và các loại ma trận đặc biệt.
    - Nếu ma trận không chéo hóa được (thiếu độc lập tuyến tính vector riêng), kết quả có thể không đúng cho việc chéo hóa.
    - Các hàm này còn hỗ trợ cho ma trận phức, ma trận Hermite, v.v.

In [ ]:
import scipy as sc
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])
B, C = sc.linalg.eig(A)
print(B)
print(C)

[5.+0.j 1.+0.j 5.+0.j]
[[ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]



- **`scipy.linalg.schur`**:
 - Hàm này dùng **Schur decomposition** (phân rã Schur) của một ma trận vuông bất kỳ, kể cả khi ma trận đó không chéo hóa được.  
  Với $A \in \mathbb{C}^{n \times n}$, ta có thể tìm được $A = Q T Q^H$, trong đó:
    - $Q$ là ma trận trực giao
    - $T$ là ma trận tam giác trên
    - $Q^H $ là ma trận liên hợp chuyển vị $(Q^H = \overline{Q}^{T})$



In [ ]:
import scipy as sc
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])
B, C = sc.linalg.schur(A)
print(B)
print(C)

[[5. 0. 0.]
 [0. 1. 0.]
 [0. 0. 5.]]
[[ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


# III. Một số ứng dụng của chéo hóa ma trận
## **1. Tính lũy thừa ma trận**
- Cho $A$ là ma trận vuông cấp $n$ hệ số thực và $A$ chéo hóa được.
- Vì $A$ chéo hóa được nên tồn tại ma trận đường chéo $D$ và ma trận khả nghịch $P$ sao cho
\begin{equation}
  A = PDP^{-1}
\end{equation}
Giả sử,
\begin{equation}
  D = diag(λ_1, λ_2, \dots, λ_n)
\end{equation}
Khi đó ta có
\begin{equation}
  A^k = (PDP^{-1})^k = PD^kP^{-1} = P\space diag(λ_1^k, λ_2^k, \dots, λ_n^k)\space P^{-1}
\end{equation}

##**2.Tìm dãy số thỏa công thức truy hồi**
***Ví dụ:*** Tính số hạng $F_n$ của dãy Fibonacci

Đặt
\begin{equation}
  u_k :=
\begin{pmatrix}
  F_{k+1} \\
  F_k
\end{pmatrix}
\end{equation} và
\begin{equation}
   A :=
\begin{pmatrix}
  1 & 1 \\
  1 & 0
\end{pmatrix}
\end{equation}
Khi đó ta có $u_{k+1} = Au_k$ <br>
Từ đó ta suy ra
\begin{equation}
u_k = A^ku_0, \space u_0=
\begin{pmatrix}
    1 \\
    0
  \end{pmatrix}
\end{equation}
Ta tính được

\begin{equation}
\begin{pmatrix}
  F_{k+1} \\
  F_k
\end{pmatrix} =
A^ku_0 = \frac{1}{λ_1 -λ_2 }
\begin{pmatrix}
 λ_1^{k+1} - λ_2^{k+1} \\
 λ_1^{k} - λ_2^{k}
\end{pmatrix}
\end{equation}
với $λ_1 = \frac{1 + \sqrt{5}}{2}$ và $λ_2 = \frac{1 - \sqrt{5}}{2}$ <br>
Từ đó ta có
\begin{equation}
F_k = \frac{1}{\sqrt{5}}\space(λ_1^k - λ_2^k), \space ∀k=\overline{1,\dots,n}
\end{equation}

##  **3. Phân tích thành phần chính (PCA) trong xử lý dữ liệu lớn**
Trong học máy, dữ liệu thường có số chiều rất lớn (nhiều thuộc tính). PCA sử dụng chéo hóa ma trận hiệp phương sai để tìm các trục chính (vector riêng ứng với trị riêng lớn nhất), giúp:
- Giảm chiều dữ liệu mà vẫn giữ lại phần lớn thông tin quan trọng.
- Loại bỏ nhiễu và trùng lặp, tối ưu hóa cho các thuật toán học máy sau đó.

- **Ví dụ:** Nhận diện khuôn mặt, xử lý ảnh, nhận diện tiếng nói.

## **4.Giải hệ phương trình vi phân tuyến tính**

Trong vật lý, hóa học, sinh học, tài chính,... nhiều hệ động học được mô tả bởi phương trình vi phân tuyến tính bậc nhất:
\begin{equation}
\vec{x}'(t) = A\vec{x}(t)
\end{equation}
Giải tổng quát của hệ này phụ thuộc vào trị riêng và vector riêng của $A$.

**Chéo hóa đơn giản hóa lời giải:**
- Nếu $A$ chéo hóa được: $A = P D P^{-1}$ thì $\vec{x}(t) = P e^{D t} P^{-1} \vec{x}(0)$, trong đó $e^{D t}$ là ma trận chéo với phần tử $i$ là $e^{\lambda_i t}$.
- Như vậy, từ hệ nhiều phương trình ràng buộc phức tạp, ta chỉ còn phải giải các phương trình vi phân bậc nhất riêng rẽ (từng trị riêng).
- Phân tích này giúp hiểu rõ bản chất động lực của hệ: mỗi trị riêng quyết định tốc độ phát triển, tắt dần hoặc dao động của một chế độ trong hệ.

**Ứng dụng:**  
- Nghiên cứu động lực học dân số, quá trình phản ứng hóa học, lan truyền dịch bệnh, biến động giá tài chính, phân rã hạt nhân, v.v.

# IV. Tham khảo

- [1] Bai, Z., Demmel, J., Dongarra, J., Ruhe, A., & van der Vorst, H. (2000). *Numerical Methods for Large Eigenvalue Problems* (2nd ed.). SIAM.
- [numpy.linalg.eig](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html)
- [scipy.linalg.eig](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html)